# imports

In [2]:
# --- Repo bootstrap (adds folder containing 'modules' to sys.path)
import os, sys

def _add_repo_root():
    cand = os.getcwd()
    root = None
    for _ in range(8):  # walk up to 8 parents
        if os.path.isdir(os.path.join(cand, "modules")):
            root = cand
            break
        cand = os.path.dirname(cand)

    if root is None:
        env = os.getenv("PROJECT_ROOT")
        if env and os.path.isdir(os.path.join(env, "modules")):
            root = env

    if root is None:
        raise SystemExit(
            "❌ Could not find repo root (folder containing 'modules'). "
            "Set PROJECT_ROOT env var or open the notebook from the repo."
        )

    if root not in sys.path:
        sys.path.insert(0, root)
    print("✓ Using repo root:", root)

_add_repo_root()


✓ Using repo root: c:\Users\felipeproenca\Documents\workspaces\personal\carbon-footprint


In [4]:
# --- Load official BR ports dataset from repo
import os, json, importlib

# If you already have these imports above, it's fine to re-import
from modules.cabotage import load_ports, find_nearest_port
from modules.addressing import resolver
from modules.road.ors_client import ORSConfig, ORSClient

def _repo_root():
    cand = os.getcwd()
    for _ in range(8):
        if os.path.isdir(os.path.join(cand, "modules")):
            return cand
        cand = os.path.dirname(cand)
    env = os.getenv("PROJECT_ROOT")
    if env and os.path.isdir(os.path.join(env, "modules")):
        return env
    raise SystemExit("❌ Could not locate repo root (folder containing 'modules').")

ROOT = _repo_root()

# Preferred path (what you asked for) + a compatible fallback
CANDIDATES = [
    os.path.join(ROOT, "modules", "cabotage", "_data", "ports_br.json"),
    os.path.join(ROOT, "modules", "cabotage_data", "ports_br.json"),
]

ports_json = next((p for p in CANDIDATES if os.path.exists(p)), None)
if not ports_json:
    raise SystemExit(
        "❌ ports_br.json not found. Expected at:\n  - "
        + "\n  - ".join(CANDIDATES)
    )

ports = load_ports(ports_json)
print(f"✓ Loaded {len(ports)} ports from {ports_json}")


✓ Loaded 28 ports from c:\Users\felipeproenca\Documents\workspaces\personal\carbon-footprint\modules\cabotage\_data\ports_br.json


In [5]:
# --- Nearest-port sanity (gate-aware if present)
def jprint(obj): 
    print(json.dumps(obj, ensure_ascii=False, indent=2, sort_keys=True))

cfg = ORSConfig(default_country="BR")
ors = ORSClient(cfg)

def _show_best(origin_text: str):
    pt = resolver.resolve_point(origin_text, ors=ors)
    best = find_nearest_port(pt["lat"], pt["lon"], ports)
    gate = best.get("gate")
    print(f"\nOrigin: {origin_text}")
    print(f"Resolved: ({pt['lat']:.6f},{pt['lon']:.6f}) — {pt['label']}")
    print(f"Nearest port: {best['name']} ({best['city']}-{best['state']}) → {best['distance_km']:.2f} km")
    if gate:
        print(f"  via gate: {gate['label']} @ ({gate['lat']:.6f},{gate['lon']:.6f})")

for origin_text in [
    "São Paulo, SP",                    # → Santos (likely via Ponta da Praia)
    "Copacabana, Rio de Janeiro, RJ",  # → Rio de Janeiro
    "Curitiba, PR",                    # → Paranaguá
]:
    _show_best(origin_text)


[12:47:18] INFO cabosupernet.road.ors_client | GEOCODE text='São Paulo, SP' country=BR size=1
[12:47:18] INFO cabosupernet.road.ors_client | GEOCODE text='Copacabana, Rio de Janeiro, RJ' country=BR size=1



Origin: São Paulo, SP
Resolved: (-23.570533,-46.663713) — São Paulo, Brazil
Nearest port: Santos (Santos-SP) → 54.88 km
  via gate: Ponta da Praia @ (-23.960800,-46.333600)


[12:47:23] INFO cabosupernet.road.ors_client | GEOCODE text='Curitiba, PR' country=BR size=1



Origin: Copacabana, Rio de Janeiro, RJ
Resolved: (-22.976478,-43.187679) — Copacabana, Rio de Janeiro, Brazil
Nearest port: Rio de Janeiro (Rio de Janeiro-RJ) → 8.68 km

Origin: Curitiba, PR
Resolved: (-25.459935,-49.280018) — Curitiba, PR, Brazil
Nearest port: Paranaguá (Paranaguá-PR) → 78.18 km


In [6]:
# --- Batch smoke: common origins → nearest port printout
origins = [
      "São Paulo, SP"
    , "Campinas, SP"
    , "Sorocaba, SP"
    , "Uberlândia, MG"
    , "Belo Horizonte, MG"
    , "Vitória, ES"
    , "Brasília, DF"
    , "Goiânia, GO"
    , "Curitiba, PR"
    , "Florianópolis, SC"
    , "Porto Alegre, RS"
    , "Fortaleza, CE"
    , "Recife, PE"
    , "Salvador, BA"
]

for o in origins:
    pt  = resolver.resolve_point(o, ors=ors)
    best = find_nearest_port(pt["lat"], pt["lon"], ports)
    g = best.get("gate")
    via = f" via {g['label']}" if g else ""
    print(f"{o:28s} → {best['name']} ({best['city']}-{best['state']}){via}  [{best['distance_km']:.1f} km]")


[12:51:27] INFO cabosupernet.road.ors_client | GEOCODE text='São Paulo, SP' country=BR size=1
[12:51:27] INFO cabosupernet.road.ors_client | GEOCODE text='Campinas, SP' country=BR size=1


São Paulo, SP                → Santos (Santos-SP) via Ponta da Praia  [54.9 km]


[12:51:30] INFO cabosupernet.road.ors_client | GEOCODE text='Sorocaba, SP' country=BR size=1


Campinas, SP                 → Santos (Santos-SP) via Ponta da Praia  [141.7 km]


[12:51:30] INFO cabosupernet.road.ors_client | GEOCODE text='Uberlândia, MG' country=BR size=1


...

Porto Alegre, RS             → Rio Grande (Rio Grande-RS)  [231.9 km]


[12:51:34] INFO cabosupernet.road.ors_client | GEOCODE text='Recife, PE' country=BR size=1


Fortaleza, CE                → Fortaleza (Mucuripe) (Fortaleza-CE)  [7.8 km]


[12:51:34] INFO cabosupernet.road.ors_client | GEOCODE text='Salvador, BA' country=BR size=1


Recife, PE                   → Recife (Recife-PE)  [5.6 km]
Salvador, BA                 → Salvador (Salvador-BA)  [10.0 km]
